## Initialize

### Importing Libraries

In [1]:
import gzip
import numpy as np
import pickle
import tensorflow as tf

from time import time

### Defining Constants

In [0]:
base_path = '.'

### Mounting File System

In [3]:
from google.colab import drive
drive.mount('/content/drive')

base_path = './drive/My Drive/CoE202TermProject'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading Input Vectors

In [0]:
with gzip.open(base_path + '/datasets/train.chunk.pickle', 'rb') as f:
    train_data = pickle.load(f)

with gzip.open(base_path + '/datasets/train.image.feats.pickle', 'rb') as f:
    train_images = pickle.load(f)

train_features = train_data['features'][:]
train_labels = train_data['labels'][:]


with gzip.open(base_path + '/datasets/valid.chunk.pickle', 'rb') as f:
    valid_data = pickle.load(f)

with gzip.open(base_path + '/datasets/valid.image.feats.pickle', 'rb') as f:
    valid_images = pickle.load(f)

valid_features = valid_data['features'][:]
valid_labels = valid_data['labels'][:]

## Defining Model

### Defining Hyperparameters

In [0]:
batch_size = 512
lr = 1e-4
n_epochs = 3
gru_units = 256
embd_size  = 1024

### Defining Constants

In [0]:
voca_size  = 100001
valid_freq = 3
seqlen_model = len(train_features[0][1])
seqlen_product = len(train_features[0][3])
seqlen_image = len(train_images[0])
n_classes  = np.max(train_labels) + 1

### Placeholders

In [0]:
input_brands = tf.placeholder(
    dtype=tf.float32, shape=(None, ), name='brands'
)

input_models = tf.placeholder(
    dtype=tf.float32, shape=(None, seqlen_model), name='input_models'
)

"""
input_model_counts = tf.placeholder(
    dtype=tf.float32, shape=(None, seqlen_model), name='input_model_counts'
)
"""

input_makers = tf.placeholder(
    dtype=tf.float32, shape=(None, ), name='makers'
)

input_products = tf.placeholder(
    dtype=tf.float32, shape=(None, seqlen_product), name='input_products'
)

"""
input_product_counts = tf.placeholder(
    dtype=tf.float32, shape=(None, seqlen_product), name='input_product_counts'
)
"""

input_prices = tf.placeholder(
    dtype=tf.float32, shape=(None, ), name='input_prices'
)

input_images = tf.placeholder(
    dtype=tf.float32, shape=(None, 2048), name='input_images'
)

labels = tf.placeholder(
    dtype=tf.int32, shape=(None, ), name='labels'
)

is_train = tf.placeholder(
    dtype=tf.bool, name='is_train'
)

### Deep neural network

In [8]:
from tensorflow.keras.layers import Activation, BatchNormalization, \
                                    Bidirectional, Concatenate, Dense, \
                                    Embedding, Flatten, GRU, Reshape

# Defining Embedding Layer
embd = Embedding(voca_size, embd_size, name='embd')

# Processing Brands
x_brands = Reshape((1, ))(input_brands)
x_brands = embd(x_brands)
x_brands = Dense(256)(x_brands)
x_brands = BatchNormalization()(x_brands, training=is_train)
x_brands = Activation('relu')(x_brands)
x_brands = Flatten()(x_brands)

# Processing Models
x_models = embd(input_models)
x_models = Bidirectional(GRU(
    gru_units, recurrent_activation='sigmoid', reset_after=True
))(x_models)

# Processing Makers
x_makers = Reshape((1, ))(input_makers)
x_makers = embd(x_makers)
x_makers = Dense(256)(x_makers)
x_makers = BatchNormalization()(x_makers, training=is_train)
x_makers = Activation('relu')(x_makers)
x_makers = Flatten()(x_makers)

# Processing Products
x_products = embd(input_products)
x_products = Bidirectional(GRU(
    gru_units, recurrent_activation='sigmoid', reset_after=True
))(x_products)

# Processing Long Texts
long_text_concat = Concatenate(axis=1)([x_models, x_products])
long_text_hidden = Dense(1024)(long_text_concat)
long_text_hidden = BatchNormalization()(long_text_hidden, training=is_train)
long_text_hidden = Activation('relu')(long_text_hidden)

# Processing Texts
text_concat = Concatenate(axis=1)([x_brands, x_makers, long_text_hidden])
text_flatten = Flatten()(text_concat)
text_hidden = Dense(n_classes)(long_text_concat)
text_hidden = BatchNormalization()(text_hidden, training=is_train)
text_hidden = Activation('relu')(text_hidden)

# Processing Images
x_images = Dense(n_classes * 2)(input_images)
x_images = BatchNormalization()(x_images, training=is_train)
x_images = Activation('relu')(x_images)

image_hidden = Dense(n_classes)(x_images)
image_hidden = BatchNormalization()(image_hidden, training=is_train)
image_hidden = Activation('relu')(image_hidden)

# Normalizing Prices
x_prices = Reshape((1, ))(input_prices)
x_prices = BatchNormalization()(x_prices, training=is_train)

# Concatenating Images and Prices
x_concat = Concatenate(axis=1)([x_products, x_images, x_prices])

# Hidden Layer
hidden1 = Dense(n_classes)(x_concat)
hidden1 = BatchNormalization()(hidden1, training=is_train)
hidden1 = Activation('relu')(hidden1)

# Output
logits = Dense(n_classes)(hidden1)

# Softmax
preds = Activation('softmax', name='predictions')(logits)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Loss funciton & Optimizer

In [9]:
# Softmax cross entropy loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(labels, n_classes), logits=logits)

# Weight decay
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([loss] + reg_losses, name='total_loss')

# Optimizer
optm = tf.train.AdamOptimizer(lr)
train_op = optm.minimize(loss, global_step=tf.train.get_global_step(), name='step_update')
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
train_op = tf.group([train_op, update_ops])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Accuracy

In [0]:
top1_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=1)
top1_acc = tf.identity(top1_acc, name='top1_acc')

top5_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=5)
top5_acc = tf.identity(top5_acc, name='top5_acc')

### Batch generator

In [0]:
def generator(mode='training'):
    if mode == 'training':
        features = train_features
        labels = train_labels
        images = train_images

    elif mode == 'valid':
        features = valid_features
        labels = valid_labels
        images = valid_images
    
    elif mode == 'test':
        features = test_features
        labels = None
        images = test_images

    n_data = len(features)
    indices = np.arange(n_data)
    np.random.shuffle(indices)
    
    for start_idx in range(0, n_data, batch_size):
        if start_idx + batch_size <= n_data:
            excerpt = indices[start_idx: start_idx + batch_size]

            if labels is not None:
                yield list(zip(*features[excerpt, :])), \
                    images[excerpt, :], \
                    labels[excerpt]
            
            else:
                yield list(zip(*features[excerpt, :])), \
                    images[excerpt, :]


### Training session

In [12]:
import tqdm
import math

with tf.Session() as sess:
    tic = time()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        print(
            "\n\nEpoch {0:03d} / {1:03d}\n"
            .format(epoch, n_epochs)
        )

        training_loss = []

        for b_features, b_images, b_label in tqdm.tqdm(
            generator(mode = 'training'),
            total = math.ceil(len(train_labels) / batch_size)
        ):
            feed_dict = {
                input_brands: b_features[0],
                input_models: b_features[1],
                input_makers: b_features[2],
                input_products: b_features[3],
                input_prices: b_features[4],
                input_images: b_images,
                labels: b_label,
                is_train: True
            }
            
            _, train_loss = sess.run(
                [train_op, loss],
                feed_dict=feed_dict
            )

            training_loss.append(train_loss)

        toc = time()
        print(
            "[*] TRAIN Loss {0:.4f} | Time {1:.2f}s"
            .format(np.mean(training_loss), toc - tic)
        )
        
        if (epoch + 1) % valid_freq == 0:
            top_1, top_5 = [], []

            for b_features, b_images, b_label in generator(mode = 'valid'):
                feed_dict = {
                    input_brands: b_features[0],
                    input_models: b_features[1],
                    input_makers: b_features[2],
                    input_products: b_features[3],
                    input_prices: b_features[4],
                    input_images: b_images,
                    labels: b_label,
                    is_train: False
                }
                    
                t1_acc, t5_acc = sess.run(
                    [top1_acc, top5_acc],
                    feed_dict=feed_dict
                )

                top_1.append(t1_acc)
                top_5.append(t5_acc)

            print(
                "[*] VALIDATION Top-1 Acc: {0:.4f} | Top-5 Acc: {1:.4f}"
                .format(np.mean(top_1), np.mean(top_5))
            )

    saver.save(sess, './drive/My Drive/CoE202TermProject/models/models')

  0%|          | 0/1563 [00:00<?, ?it/s]



Epoch 000 / 003



  0%|          | 1/1563 [00:00<03:57,  6.56it/s]

[*] TRAIN Loss 1.9153 | Time 218.75s


Epoch 001 / 003



  0%|          | 1/1563 [00:00<03:55,  6.64it/s]

[*] TRAIN Loss 0.8257 | Time 425.24s


Epoch 002 / 003



100%|█████████▉| 1562/1563 [03:26<00:00,  7.56it/s]


[*] TRAIN Loss 0.5518 | Time 631.90s
[*] VALIDATION Top-1 Acc: 0.3179 | Top-5 Acc: 0.8401


## Test process
### Load the test data

In [0]:
# test data
with gzip.open('./drive/My Drive/CoE202TermProject/datasets/test.chunk.pickle', 'rb') as f:
    test_data = pickle.load(f)

with gzip.open('./drive/My Drive/CoE202TermProject/datasets/valid.image.feats.pickle', 'rb') as f:
    test_images = pickle.load(f)

test_features  = test_data['features'][:]
pids = test_data['pids'][:]

### Batch generator for test dataset

In [0]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./drive/My Drive/CoE202TermProject/models/models.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./drive/My Drive/CoE202TermProject/models/'))
    DNN = tf.get_default_graph()

    preds = []
    for b_features, b_images in generator(mode='test'):
        feed_dict = {
            DNN.get_tensor_by_name('input_brands:0'): b_features[0],
            DNN.get_tensor_by_name('input_models:0'): b_features[1],
            DNN.get_tensor_by_name('input_makers:0'): b_features[2],
            DNN.get_tensor_by_name('input_products:0'): b_features[3],
            DNN.get_tensor_by_name('input_prices:0'): b_features[4],
            DNN.get_tensor_by_name('input_images:0'): b_images,
            DNN.get_tensor_by_name('is_train:0'): False
        }

        pred = sess.run(DNN.get_tensor_by_name('predictions:0'), feed_dict=feed_dict)
        preds.extend(pred)


### Save the submission files

In [0]:
# Indexing of predictions
argpreds = np.argmax(preds, axis=1)

# Load label dictionary
with open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle', 'rb') as f:
    y_dict = pickle.load(f)
# y_dict = pickle.loads(open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle').read())

# Inverse label dictionary
inv_y_dict = dict((y,x) for x,y in y_dict.items())
submissions = [inv_y_dict[argpred] for argpred in argpreds]

# Write the results to 'submissions.csv'
f = open('./drive/My Drive/CoE202TermProject/submissions.csv', 'w')
for i, j in zip(pids, submissions):
    line = '{},{}\n'.format(i,j)
    f.write(line)
f.close()

You should submit the 'submissions.csv' file and 4 tf.save files ('checkpoint', 'dnn_models.data', 'dnn_models.index', 'dnn_models.meta') in models folder